In [0]:
%sql
--To check if there are duplicates in table contactos before the join , just for security

SELECT
  id,
  data_criacao_da_lead,
  COUNT(*) AS cnt
FROM sc_gold.contactos_pbs
GROUP BY id, data_criacao_da_lead
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, data_criacao_da_lead;


In [0]:
%sql
DESCRIBE sc_gold.`leads_pbs`;
-- or
SHOW COLUMNS IN sc_gold.`leads_pbs`;


In [0]:
%sql
SELECT
  created_time AS lead_created_time
FROM sc_silver.leads_pbs
LIMIT 50;


In [0]:
%sql
SELECT
  data_criacao_da_lead
FROM sc_silver.contactos_pbs
LIMIT 50;

In [0]:

%sql

SELECT * FROM sc_gold.campaigns             LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.leads_pbs             LIMIT 50;


In [0]:
%sql

SELECT * FROM sc_gold.contactos_pbs         LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.propostas_realizadas  LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.deals                 LIMIT 50;

In [0]:
%sql
SELECT * FROM sc_gold.contactos_pbs         LIMIT 50;

In [0]:
# Build and run: CREATE OR REPLACE TABLE sc_gold.contactos_pbs_2 AS SELECT ... FROM sc_gold.contactos_pbs

# 1) Read source columns in order
cols = spark.sql("""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'contactos_pbs'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns you do NOT want renamed (e.g., keep id as-is)
skip = set()   # e.g., {'id'}

# 2) Build the select list with aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_contactos') or c in skip) else f"{c}_contactos"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build the full CREATE TABLE AS SELECT statement
create_sql = f"""
CREATE OR REPLACE TABLE sc_gold.contactos_pbs_2 AS
SELECT {select_sql}
FROM sc_gold.contactos_pbs
"""

print(create_sql)  # shows exactly what will run
spark.sql(create_sql)

# 4) Quick verification
spark.sql("DESCRIBE TABLE sc_gold.contactos_pbs_2").show(truncate=False)
spark.sql("SELECT * FROM sc_gold.contactos_pbs_2 LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.contactos_pbs_2

In [0]:
# Source and target
source_table = "sc_gold.propostas_realizadas"
target_table = "sc_gold.propostas_realizadas2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'propostas_realizadas'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged
skip = set()   # e.g., {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_propostas') or c in skip) else f"{c}_propostas"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Show what will run
spark.sql(create_sql)

# 4) Verify
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.propostas_realizadas2

In [0]:
# Source and target
source_table = "sc_gold.deals"
target_table = "sc_gold.deals2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'deals'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged (e.g., primary keys)
skip = set()   # Example: {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_deals') or c in skip) else f"{c}_deals"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Show generated SQL
spark.sql(create_sql)

# 4) Verify structure and sample data
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.deals2

In [0]:
# Source and target
source_table = "sc_gold.campaigns"
target_table = "sc_gold.campaigns2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'campaigns'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged (like primary keys)
skip = set()   # Example: {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_campaigns') or c in skip) else f"{c}_campaigns"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Preview generated SQL
spark.sql(create_sql)

# 4) Verify new table
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.campaigns2

In [0]:
%sql
SELECT COUNT(*) AS strict_matches
FROM sc_gold.leads_pbs l
JOIN sc_gold.contactos_pbs_2 c
  ON  l.converted_contact = c.id_contactos
  AND l.created_time      = c.data_criacao_da_lead_contactos;


In [0]:
%sql
WITH joined AS (
  SELECT
    l.*,
    c.*
  FROM sc_gold.leads_pbs       AS l
  JOIN sc_gold.contactos_pbs_2 AS c
    ON  l.converted_contact = c.id_contactos
   AND l.created_time      = c.data_criacao_da_lead_contactos
)
SELECT COUNT(*) AS check_count FROM joined;   -- should be ~118329


In [0]:
%sql
-- (optional) ensure you're in the right place
-- USE CATALOG workspace;
-- USE SCHEMA sc_gold;

CREATE OR REPLACE TABLE sc_gold.Features_Table
USING DELTA
AS
SELECT
  l.*,
  c.*
FROM sc_gold.leads_pbs       AS l
JOIN sc_gold.contactos_pbs_2 AS c
  ON  l.converted_contact = c.id_contactos
  AND l.created_time      = c.data_criacao_da_lead_contactos;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  f.*,
  p.*   -- assumes Propostas columns are already suffixed (e.g., *_propostas)
FROM sc_gold.Features_Table AS f
LEFT JOIN sc_gold.propostas_realizadas2 AS p
  ON f.id_contactos = p.id_contacto_propostas;   -- keep all Feature rows


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  f.*,
  d.*   -- assumes Deals columns are already suffixed (e.g., *_deals)
FROM sc_gold.Features_Table AS f
LEFT JOIN sc_gold.deals2 AS d
  ON f.id_contacto_propostas = d.id_contacto_deals;   -- keep all Feature rows


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  f.*,
  cp.*   -- assumes campaigns columns are already suffixed (e.g., *_campaigns)
FROM sc_gold.Features_Table AS f
LEFT JOIN sc_gold.campaigns2 AS cp
  ON f.campanha_deals = cp.id_campaigns;   -- join via Features_Table.campanha_deals -> campaigns.id_campaigns


In [0]:
%%sql
CREATE OR REPLACE TABLE sc_gold.fact_table AS
SELECT
    l.id,
  l.converted_contact,
  l.created_time,
  c.id_contactos,
  c.data_criacao_da_lead_contactos
FROM sc_gold.leads_pbs AS l
JOIN sc_gold.contactos_pbs_2 AS c
  ON l.converted_contact = c.id_contactos
 AND l.created_time = c.data_criacao_da_lead_contactos;


In [0]:
%%sql
CREATE OR REPLACE TABLE sc_gold.fact_table AS
SELECT
  f.*,
 
    p.id_contacto_propostas,
  p.id_proposta_realizada_propostas

FROM sc_gold.fact_table AS f
LEFT JOIN sc_gold.propostas_realizadas2 AS p
  ON p.id_contacto_propostas = f.id_contactos;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Fact_table AS
SELECT
  f.*,
  d.id_deals,
  d.id_contacto_deals,
  d.campanha_deals
FROM sc_gold.Fact_table AS f
LEFT JOIN sc_gold.deals2 AS d
  ON f.id_contacto_propostas = d.id_contacto_deals;


In [0]:
# snapshot the current table to a temp view
spark.sql("CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM sc_gold.fact_table")

to_drop = {'converted_contact','data_criacao_da_lead_contactos','id_contacto_propostas','id_contacto_deals'}
cols = [f"`{c}`" for c in spark.table("sc_gold.fact_table").columns if c not in to_drop]

spark.sql(f"""
CREATE OR REPLACE TABLE sc_gold.fact_table
USING DELTA AS
SELECT {', '.join(cols)} FROM _src
""")

spark.sql("DROP VIEW _src")


In [0]:
%sql
-- 1) Summary counts of duplicates on (id, created_time)
WITH d AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_gold.leads_pbs
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)                    AS duplicated_keys,          -- number of (id,created_time) combos with >1 row
  SUM(cnt)                    AS rows_in_duplicated_keys,  -- total rows participating in duplicates
  SUM(cnt) - COUNT(*)         AS extra_rows_to_dedup       -- rows to drop if you keep 1 per key
FROM d;

-- 2) See the duplicated keys and their sizes (top 100)
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_gold.leads_pbs
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id
LIMIT 100;


In [0]:
%sql
-- SUMMARY: how many duplicate (id, created_time) keys exist in sc_gold.features ( this is normal , can have multiple proposals)
WITH d AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_gold.features_table
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)            AS duplicated_keys,
  SUM(cnt)            AS rows_in_duplicated_keys,
  SUM(cnt) - COUNT(*) AS extra_rows_to_dedup
FROM d;


In [0]:
%sql
-- SUMMARY: how many duplicate (id_propostas, created_time) keys exist
WITH d AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_gold.propostas_realizadas
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)            AS duplicated_keys,          -- number of key pairs with >1 row
  SUM(cnt)            AS rows_in_duplicated_keys,  -- total rows across those keys
  SUM(cnt) - COUNT(*) AS extra_rows_to_dedup       -- rows you’d remove if keeping 1 per key
FROM d;

-- DETAILS: which keys are duplicated (top 100 by size)
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_gold.propostas_realizadas
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id
LIMIT 100;

-- OPTIONAL: list the actual duplicate rows
SELECT t.*
FROM sc_gold.propostas_realizadas t
JOIN (
  SELECT id, created_time
  FROM sc_gold.propostas_realizadas
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
) d
ON t.id = d.id AND t.created_time = d.created_time
ORDER BY t.id, t.created_time;


In [0]:
%sql
WITH d AS (
  SELECT
    `id`,
    `id_deals`,
    `id_proposta_realizada_propostas`,
    COUNT(*) AS cnt
  FROM sc_gold.features_table
  GROUP BY `id`, `id_deals`, `id_proposta_realizada_propostas`
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)              AS duplicated_keys,
  SUM(d.cnt)            AS rows_in_duplicated_keys,
  SUM(d.cnt) - COUNT(*) AS extra_rows_to_dedup
FROM d AS d;


In [0]:
%sql
WITH d AS (
  SELECT
    `id`,
    `id_deals`,
    `id_proposta_realizada_propostas`,
    COUNT(*) AS cnt
  FROM sc_gold.fact_table
  GROUP BY `id`, `id_deals`, `id_proposta_realizada_propostas`
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)              AS duplicated_keys,
  SUM(d.cnt)            AS rows_in_duplicated_keys,
  SUM(d.cnt) - COUNT(*) AS extra_rows_to_dedup
FROM d AS d;


In [0]:
%sql
SELECT * FROM sc_gold.Fact_table

In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
# 1) Build keep list to Ml Models
cols_to_delete = [  # the  names we listed
    "_fivetran_id","id","lead_owner","lead_owner_name","consentimento",
    "id_lead","chave_instalacao_bd_rede","chave_concessao_bd_rede","chave_contrato_concessionario_bd_rede",
    "chave_agrupamento_performance_vendas","chave_agrupamento_cliente",
    "data_de_validade_de_consentimento_hyundai","lead_response_time","converted_date_time",
    "_fivetran_deleted","_fivetran_synced","_fivetran_id_contactos","id_contactos",
    "contacto_owner_contactos","contacto_owner_name_contactos","created_time_contactos",
    "id_contacto_contactos","consentimento_contactos","id_lead_conversao_contactos",
    "data_de_recolha_de_consentimento_hyundai_contactos","chave_concessao_bd_rede_contactos",
    "chave_instalacao_bd_rede_contactos","chave_contrato_concessionario_bd_rede_contactos",
    "chave_agrupamento_performance_vendas_contactos","chave_agrupamento_cliente_contactos",
    "link_centro_consentimento_contactos","_fivetran_deleted_contactos","_fivetran_synced_contactos",
    "_fivetran_id_propostas","id_propostas","created_time_propostas",
    "proposta_realizada_owner_propostas","proposta_realizada_owner_name_propostas",
    "modified_time_propostas","conta_name_propostas","id_contacto_propostas","contacto_name_propostas",
    "deal_name_propostas","created_by_propostas","modified_by_propostas","descricao_apoio_propostas",
    "codigo_da_proposta_propostas","descricao_pintura_propostas","data_de_criacao_da_proposta_propostas",
    "data_de_entrega_da_proposta_propostas","id_conta_propostas","id_negocio_propostas",
    "id_proposta_realizada_propostas","data_de_validade_da_proposta_propostas","codigo_modelo_propostas",
    "sufixo_modelo_propostas","codigo_cor_exterior_propostas","codigo_cor_interior_propostas",
    "concessionario_owner_propostas","data_prevista_matricula_propostas","data_prevista_de_entrega_propostas",
    "data_da_conclusao_propostas","instalacao_propostas","chave_instalacao_bd_rede_propostas",
    "chave_concessao_bd_rede_propostas","cod_proposta_propostas","codigo_fabricante_cor_interior_propostas",
    "codigo_fabricante_cor_exterior_propostas","_fivetran_deleted_propostas","_fivetran_synced_propostas",
    "_fivetran_id_deals","deal_owner_name_deals","id_contacto_deals",
    "closing_date_deals","deal_owner_deals","conta_name_deals","contacto_name_deals","id_negocio_deals",
    "data_prevista_de_reentrada_em_negocio_deals","id_conta_deals","codigo_negocio_deals","data_venda_deals",
    "nome_contacto_deals","id_deals","data_decisao_negocio_deals","_fivetran_deleted_deals",
    "_fivetran_synced_deals","_fivetran_id_campaigns","id_campaigns",
    "campaign_owner_name_campaigns","created_time_campaigns","codigo_campanha_campaigns",
    "_fivetran_deleted_campaigns","_fivetran_synced_campaigns"
]

src_tbl = "sc_gold.Features_Table"

# compute keep list from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # quote names safely

# 2) Snapshot and recreate from projection
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")
spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")
spark.sql("DROP VIEW _src")

#You should generally remove columns that:
#Are unique identifiers (e.g., _fivetran_id, id, id_lead, id_contactos, id_propostas, id_deals, id_campaigns).
#Are timestamps or sync markers from ETL (_fivetran_synced, _fivetran_deleted, etc.).
#Contain names of people or owners (lead_owner_name, campaign_owner_name_campaigns, etc.).
#Are free-text codes or descriptive identifiers (codigo_campanha_campaigns, codigo_da_proposta_propostas, etc.).
#Are internal foreign keys / lookup fields (chave_instalacao_bd_rede, chave_agrupamento_cliente, etc.).
#Are emails / links (link_centro_consentimento_contactos).




In [0]:
# 1) Built the second delete list after a better analysis 
cols_to_delete = [
    # --- timestamps / dates ---
  
    "data_criacao_da_lead_contactos",
    "data_prevista_de_entrega_deals",
    "data_prevista_matricula_deals",
    "data_criacao_da_lead_deals",

    # --- IDs / keys ---
    "created_by", "modified_by",
    "id_classe_propostas", "id_model_group_propostas",
    "id_combustivel_propostas", "id_modelo_propostas",
    "chave_agrupamento_cliente_propostas",
    "tylacode_propostas",
    "chave_concessao_bd_rede_deals", "chave_instalacao_bd_rede_deals",
    "chave_contrato_concessionario_bd_rede_deals",
    "chave_agrupamento_performance_vendas_deals", "chave_agrupamento_cliente_deals","campanha_deals",

    # --- descriptive text ---
    "descricao_model_group_propostas",
    

    # --- administrative fields ---
     "gestor_area_propostas",

    # --- duplicated / rarely useful flags ---
    "hubleads__nao_comunica_com_cc_",
    "converted_contact", "converted_deal", "converted_from_lead_deals",
    "ccupdated_contactos", "myhyundai_contactos",
    "f2_deals", "modelovendaupdate_deals"
]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# 2) Snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN month(created_time) IN (12, 1, 2) THEN 'Winter'
    WHEN month(created_time) IN (3, 4, 5) THEN 'Spring'
    WHEN month(created_time) IN (6, 7, 8) THEN 'Summer'
    ELSE 'Autumn'
  END AS Season
FROM sc_gold.Features_Table;

--I created a Season for ML Models ( thinking in the prediction already)


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  datediff(created_time_deals, created_time) AS lead_to_deal_days
FROM sc_gold.Features_Table;

-- this is deals date- leads date to give the number of days that take , for my surprise there are negatives but can be typos or retroactive data entry → A deal was created manually before the lead was officially entered in the CRM.


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN datediff(created_time_deals, created_time) <= 7 THEN 'Fast'
    WHEN datediff(created_time_deals, created_time) <= 30 THEN 'Medium'
    ELSE 'Slow'
  END AS deal_speed_category
FROM sc_gold.Features_Table;



In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN created_time_deals < created_time THEN 1
    ELSE 0
  END AS deal_before_lead
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN dayofweek(created_time) IN (1,7) THEN 1
    ELSE 0
  END AS is_weekend_lead
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN day(created_time_deals) >= 25 THEN 1
    ELSE 0
  END AS is_month_end_deal
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table
SELECT 
    *,
    CASE 
        WHEN qtd_viaturas_deals = 0 OR qtd_viaturas_deals IS NULL 
        THEN 0
        ELSE num_de_propostas_deals / qtd_viaturas_deals
    END AS proposals_per_vehicle
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  year(created_time)  AS lead_year,
  month(created_time) AS lead_month
FROM sc_gold.Features_Table;


In [0]:
cols_to_delete = [
    "created_time",
    "activities_involved",
    "created_time_deals"
]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")


In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
%sql
SELECT
  is_converted,
  COUNT(*) AS total_leads
FROM sc_gold.leads_pbs
GROUP BY is_converted
ORDER BY is_converted;



In [0]:
# --- K-Means via pandas + scikit-learn (version-safe for OneHotEncoder) ---
import numpy as np
import pandas as pd
import sklearn
from packaging.version import parse as V

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score



In [0]:
# Pull data (sample if your table is very large)
pdf = spark.sql("SELECT * FROM sc_gold.Features_Table").toPandas()
# Example sampling to avoid driver OOM:
# pdf = spark.sql("SELECT * FROM sc_gold.Features_Table TABLESAMPLE (50 PERCENT)").toPandas()

# Identify columns
cat_cols = pdf.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
num_cols = pdf.select_dtypes(include=["number"]).columns.tolist()

# Preprocessing blocks
numeric_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Version-safe OneHotEncoder kwargs
ohe_kwargs = {"handle_unknown": "ignore"}
if V(sklearn.__version__) >= V("1.2"):
    ohe_kwargs["sparse_output"] = True      # modern scikit-learn
else:
    ohe_kwargs["sparse"] = True             # older scikit-learn

categorical_tf = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(**ohe_kwargs))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_tf, num_cols),
        ("cat", categorical_tf, cat_cols),
    ],
    remainder="drop",
    sparse_threshold=1.0  # keep matrix sparse if possible
)

# Transform to model matrix
X = preprocess.fit_transform(pdf)



In [0]:
# Search k by silhouette
k_range = range(2, 9)
scores = []
for k in k_range:
    km = KMeans(n_clusters=k, n_init=20, random_state=42)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels) if len(set(labels)) > 1 else np.nan
    scores.append((k, sil))

best_k = max(scores, key=lambda x: (-1 if np.isnan(x[1]) else x[1]))[0]
print("Best k:", best_k)
print(pd.DataFrame(scores, columns=["k","silhouette"]).sort_values("silhouette", ascending=False))

# Fit final model
km = KMeans(n_clusters=best_k, n_init=50, random_state=42)
labels = km.fit_predict(X)
pdf["cluster"] = labels

# Cluster sizes & peek
print("Cluster sizes:\n", pdf["cluster"].value_counts().sort_index())
preview_cols = [c for c in ["lead_source","lead_status","lead_status_2","is_converted","distrito","pais"] if c in pdf.columns]
print(pdf.head(10)[["cluster"] + preview_cols])

# Write back to Delta (works from a cluster notebook; may be blocked on SQL Warehouse)
try:
    spark.createDataFrame(pdf).write.mode("overwrite").format("delta").saveAsTable("sc_gold.Features_Table_kmeans")
    print("Saved table: sc_gold.Features_Table_kmeans")
except Exception as e:
    print("Could not save to Delta from this environment:", e)


In [0]:
# === Replace your cols_to_delete with this one (excludes columns you already removed) ===
cols_to_delete = [
    # ----------------------------
    # LEAKAGE (deal/post-outcome) — info not available at lead time
    # ----------------------------
    "lead_status",                    # leakage: deal-stage info
    "lead_status_2",                  # leakage: deal-stage variant
    "stage_deals",                          # leakage: pipeline stage after lead
    "forecast_type_deals",                  # leakage
    "motivo_fecho_negocio_deals",           # leakage: reason for closing
     "categoria_negocio_venda_deals",        # leakage; also mostly null in many exports
    "modelo_de_venda_deals",                # leakage; often mostly null
    "tipo_de_negocio_deals",                # leakage; often mostly null
        "tipo_de_pedido",                 # leakage
         "tipo_de_pedido_contactos",                 # leakage
           "categoria_proposta_propostas",                       # leakage
           "activities_involved_deals",            # leakage: totals up to close
 

    # ----------------------------
    # ID-like / keys (non-predictive & high-unique)
    # ----------------------------
    "contrato_concessionario_deals",        # ID-like (deal)
    "contrato_concessionario",              # ID-like
    "contrato_concessionario_contactos",    # ID-like
    "concessionario_escolhido",             # ID-like / chosen dealer
    "estado_do_pedido_propostas",           # often key-like / admin-ish

    # ----------------------------
    # Data quality: free text / high null / near-constant / high-cardinality
    # ----------------------------
    "descricao_classe_propostas",           # free text / mostly null
    "subject_propostas",                    # free text / high-cardinality
    "margem_frota_eni_propostas",           # mostly null
    "sub_total_com_extras_propostas",       # mostly null
    "valor_aprovado_propostas",             # mostly null
    "valor_extras_propostas",               # mostly null
    "valor_do_apoio_pedido_propostas",      # mostly null; also key-like in some exports
    "n__matriculas_associadas_deals"        # near-constant / mostly null (deal-level)
]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")


In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
from pyspark.sql import functions as F

tbl = "sc_gold.Features_Table"
cols = [
    "concessao",
    "instalacao",
    "agrupamento_performance_vendas",
    "agrupamento_cliente",
    "concessao_contactos",
    "instalacao_contactos",
    "agrupamento_performance_vendas_contactos",
    "agrupamento_cliente_contactos",
    "concessao_propostas",
    "concessao_deals",
    "instalacao_deals",
    "agrupamento_performance_vendas_deals",
    "agrupamento_cliente_deals",
]

df = spark.table(tbl)
present = [c for c in cols if c in df.columns]
exprs = [
    (F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c)) for c in present
]
res = df.agg(*exprs).collect()[0].asDict()
total = df.count()

out = [{"column": c, "null_count": int(res.get(c, 0)), "rows_total": total,
        "null_pct": f"{(res.get(c, 0)/total)*100:.2f}%"} for c in present]
for c in cols:
    if c not in present:
        out.append({"column": c, "null_count": "—", "rows_total": total, "null_pct": "not in table"})

spark.createDataFrame(out).orderBy("column").show(truncate=False)


In [0]:
# === Replace your cols_to_delete with this one (excludes columns you already removed) ===
cols_to_delete = [
    # ----------------------------
    # LEAKAGE (deal/post-outcome) — info not available at lead time
    # ----------------------------
    "lead_status",                    # leakage: deal-stage info
    "instalacao",
    "agrupamento_performance_vendas",
    "agrupamento_cliente",
    "instalacao_contactos",
    "agrupamento_performance_vendas_contactos",
    "agrupamento_cliente_contactos",
    "concessao_propostas",
    "concessao_deals",
    "instalacao_deals",
    "agrupamento_performance_vendas_deals",
    "agrupamento_cliente_deals" ]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")

In [0]:
%sql

--I will replace the blanks with No campaign because I really want to study this column on the model

UPDATE sc_gold.Features_Table
SET campaign_name_campaigns = 'No Campaign'
WHERE campaign_name_campaigns IS NULL;



In [0]:
%sql

--I will replace the blanks with Unknown because I really want to study this column on the model

UPDATE sc_gold.Features_Table
SET motivo_da_perda_deals = 'Unknown'
WHERE motivo_da_perda_deals IS NULL;

In [0]:
%sql
UPDATE sc_gold.Features_Table
SET modelo = 'Unknown'
WHERE modelo IS NULL;

UPDATE sc_gold.Features_Table
SET versao_propostas = 'Unknown'
WHERE versao_propostas IS NULL;

UPDATE sc_gold.Features_Table
SET combustivel_propostas = 'Unknown'
WHERE combustivel_propostas IS NULL;

UPDATE sc_gold.Features_Table
SET origem_do_negocio_deals = 'Unknown'
WHERE origem_do_negocio_deals IS NULL;


In [0]:
from pyspark.sql import functions as F

#since nulls are only 0.07%, filling with the mode (most frequent value) is a very good option.
#Fill with mean/median
#Median = more robust to skew/outliers.

col_name = "tipo_cliente_contactos"

# get mode (most frequent value)
mode_value = (
    df.groupBy(col_name)
      .count()
      .orderBy(F.desc("count"))
      .first()[0]
)

# fill nulls with mode
df = df.na.fill({col_name: mode_value})

# overwrite the existing table
df.write.mode("overwrite").saveAsTable("sc_gold.Features_Table")


In [0]:
from pyspark.sql import functions as F

#this one looks numeric (proposals_per_vehicle) with small percentage of nulls (2.53%). That’s easy to handle.


col_name = "proposals_per_vehicle"

# calculate median
median_val = df.approxQuantile(col_name, [0.5], 0.01)[0]  # 0.5 = median, tolerance=0.01

# print the median
print(f"Median value for {col_name}: {median_val}")

# fill nulls with median
df = df.na.fill({col_name: median_val})


In [0]:
from pyspark.sql import functions as F

# Load the table
df = spark.table("sc_gold.Features_Table")

# ===== Median fill for proposals_per_vehicle =====
col_name = "proposals_per_vehicle"

# calculate median with approxQuantile
median_val = df.approxQuantile(col_name, [0.5], 0.01)[0]

# fill nulls with the median
df = df.na.fill({col_name: median_val})

# ===== Overwrite the table =====
df.write.mode("overwrite").saveAsTable("sc_gold.Features_Table")


In [0]:
#pvp_propostas is the price of sales of the cars , we have the models , maybe do a replace by median of each model and when the model is Unknown replace by the overall median?
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType

TBL = "sc_gold.Features_Table"

# 0) Load
df = spark.table(TBL)

# Count nulls before
before_nulls = df.filter(F.col("pvp_propostas").isNull()).count()
print(f"Nulls in pvp_propostas BEFORE: {before_nulls:,}")

# 1) Compute per-model median
model_medians = (
    df.filter(F.col("pvp_propostas").isNotNull())
      .groupBy("modelo")
      .agg(F.expr("percentile_approx(pvp_propostas, 0.5)").alias("median_pvp"))
)

# 2) Join medians
df = df.join(model_medians, on="modelo", how="left")

# 3) Compute overall median
overall_median = df.approxQuantile("pvp_propostas", [0.5], 0.01)[0]
print(f"Overall median pvp_propostas: {overall_median}")

# 4) Impute
df = df.withColumn(
    "pvp_propostas",
    F.when(
        F.col("pvp_propostas").isNull(),
        F.when(F.col("modelo") == "Unknown", F.lit(overall_median))
         .otherwise(F.coalesce(F.col("median_pvp"), F.lit(overall_median)))
    ).otherwise(F.col("pvp_propostas"))
)

# 5) Drop helper column
df = df.drop("median_pvp")

# 6) Cast back to decimal(29,2)
df = df.withColumn("pvp_propostas", F.col("pvp_propostas").cast(DecimalType(29, 2)))

# Count nulls after
after_nulls = df.filter(F.col("pvp_propostas").isNull()).count()
print(f"Nulls in pvp_propostas AFTER: {after_nulls:,}")

# 7) Overwrite the table
df.write.mode("overwrite").saveAsTable(TBL)
print(f"Table {TBL} overwritten successfully.")

#Computes the median price per modelo.

#Fills missing pvp_propostas with that median; if the row’s modelo is "Unknown" (or the model median is unavailable), it falls back to the overall median.

#Overwrites sc_gold.Features_Table with the imputed data


In [0]:
# Technic Median imputation + missing flag (baseline for ML)
#Fill nulls with the median (robust to skew).
#Add _missing indicator columns.
#That way ML models can separate “normal duration” vs. “missing duration”.
#🔹 Why it helps ML
#If missing values are completely random, the model just uses the imputed median and ignores the flag.
#If missing values are systematic (e.g., certain customers or deals often have missing durations), then the flag lets the model capture that pattern.
#You don’t distort the data too much (everything stays numeric).
#Works with tree-based models (XGBoost, LightGBM, Random Forest) and linear models.
#Is it the best option?
#For a first baseline ML pipeline: ✅ Yes, it’s the safest, most widely used approach.
#For business-critical features like your durations: it’s a solid start, but may not be the final solution. You can improve later with group-based medians or domain-specific imputations (e.g., different medians for B2B vs B2C customers).
#Think of Option 1 as the “default robust preprocessing”. Once you see feature importance and model performance, you can refine further.

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df  = spark.table(TBL)

# Total rows
total_rows = df.count()

# Count how many rows have 1 in that column + percentage
df.filter(F.col("overall_sales_duration_deals_missing") == 1) \
  .agg(
      F.count("*").alias("count_1")
  ) \
  .withColumn("pct_1", (F.col("count_1") / total_rows) * 100) \
  .show()


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

duration_cols = ["overall_sales_duration_deals", "lead_to_deal_days"]

for c in duration_cols:
    med = df.approxQuantile(c, [0.5], 0.01)[0]
    print(f"Median for {c}: {med}")

    # add missing flag + impute
    df = df.withColumn(f"{c}_missing", F.when(F.col(c).isNull(), 1).otherwise(0).cast(IntegerType()))
    df = df.na.fill({c: med})
    df = df.withColumn(c, F.col(c).cast(IntegerType()))  # keep dtype consistent

# Overwrite and allow schema change (because of new *_missing columns)
df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(TBL)
print(f"✅ Overwrote {TBL} with median imputations + missing flags.")



In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

row_count = df.count()
dtypes = dict(df.dtypes)

# --- STRICT NULLS ONLY (what you asked) ---
null_aggs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
null_row = df.agg(*null_aggs).collect()[0].asDict()

rows = []
for c in df.columns:
    n_null = int(null_row.get(c, 0))
    rows.append((c, dtypes[c], n_null, row_count, f"{(n_null/row_count)*100:.2f}%"))

out = spark.createDataFrame(rows, ["column", "dtype", "null_count", "rows_total", "null_pct"])
display(out.orderBy(F.desc("null_pct")))


In [0]:
# === Replace your cols_to_delete with this one (excludes columns you already removed) ===
cols_to_delete = [
    # ----------------------------
    # LEAKAGE (deal/post-outcome) — info not available at lead time
    # ----------------------------
    "lead_status",  
      "apoio_total_propostas",
    "forma_de_pagamento_propostas",
    "apoio_concessionario_propostas",
    "comparticipacao_retoma_concessao_propostas",
    "apoio_retoma_hyundai_propostas",
    "apoio_financiamento_cetelem_propostas",
    "formulario_deals",
    "comparticipacao_campanha_da_concessao_propostas",
    "lead_conversion_time_deals",
    "apoio_campanha_hyundai_propostas",
    "probability_____deals",
    "distrito",
    "apoio_hyundai_portugal_propostas",
    "valor_campanha_propostas",
    "desconto_total__c__apoio_de_importador__propostas",
    "categoria_negocio_deals",
        "distrito_contactos",
    "estado_ordering_propostas",
    "tasks_involved_deals",
    "events_involved_deals",
    "calls_involved_deals",
    "desconto_total_propostas",
    "pais"                 
   ]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
CHECK_COLS = ["modelo", "modelo_contactos", "modelo_propostas", "modelos_deals"]

df = spark.table(TBL)
present = [c for c in CHECK_COLS if c in df.columns]
missing = [c for c in CHECK_COLS if c not in df.columns]

row_count = df.count()
# null counts for present columns
nulls = (df.agg(*[F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in present])
           .collect()[0]
           .asDict())

# build result rows
rows = []
for c in present:
    n_null = int(nulls.get(c, 0))
    rows.append((c, row_count, n_null, f"{(n_null/row_count)*100:.2f}%"))

out = spark.createDataFrame(rows, ["column", "rows_total", "null_count", "null_pct"])
display(out.orderBy("column"))

if missing:
    print("Not found in table (skipped):", missing)


In [0]:
# === Replace your cols_to_delete with this one (excludes columns you already removed) ===
cols_to_delete = [
    # ----------------------------
    # LEAKAGE (deal/post-outcome) — info not available at lead time
    # ----------------------------
    "modelo_contactos",
    "modelo_propostas",
    "modelos_deals", 
    "formulario",
    "tasks_only_deals",
    "events_only_deals",
    "calls_only_deals",
    "tasks_involved",
    "events_involved",
    "calls_involved",
               
   ]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

row_count = df.count()
dtypes = dict(df.dtypes)

# --- STRICT NULLS ONLY (what you asked) ---
null_aggs = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
null_row = df.agg(*null_aggs).collect()[0].asDict()

rows = []
for c in df.columns:
    n_null = int(null_row.get(c, 0))
    rows.append((c, dtypes[c], n_null, row_count, f"{(n_null/row_count)*100:.2f}%"))

out = spark.createDataFrame(rows, ["column", "dtype", "null_count", "rows_total", "null_pct"])
display(out.orderBy(F.desc("null_pct")))

In [0]:
# FIRST TRY OF KMEANS - THE RESULTS WERE NOT GOOD

# =========================
# 1) STRATIFIED SAMPLE (multi-key, exact per-stratum counts)
# =========================
from pyspark.sql import functions as F
from pyspark.sql.window import Window

SRC = "sc_gold.Features_Table"
OUT = "sc_gold.Features_Table_sampled"

# Choose desired strata; only the ones present will be used
STRATA_DESIRED = ["is_converted", "lead_source", "pais", "lead_month"]
SAMPLE_TARGET  = 30_000   # total rows you want in the sample (adjust)
MIN_PER_STRAT  = 200      # minimum rows per stratum (adjust)
SEED           = 42

df = spark.table(SRC)
present_strata = [c for c in STRATA_DESIRED if c in df.columns]

if present_strata:
    # Build a stratum key (treat NULLs as their own bucket)
    key_exprs = [F.coalesce(F.col(c).cast("string"), F.lit("∅")) for c in present_strata]
    dfk = df.withColumn("_stratum", F.concat_ws("||", *key_exprs))

    total = dfk.count()

    # Per-stratum counts + target take_n
    cnt = (dfk.groupBy("_stratum").count()
             .withColumn("take_n",
                         F.least(
                             F.col("count"),
                             F.greatest(F.lit(MIN_PER_STRAT),
                                        F.ceil(F.col("count") * F.lit(SAMPLE_TARGET) / F.lit(total)))
                         )))

    # Exact sampling per stratum (top-k by random within each stratum)
    win = Window.partitionBy("_stratum").orderBy(F.rand(SEED))
    sampled = (dfk.join(cnt, on="_stratum", how="left")
                  .withColumn("_rn", F.row_number().over(win))
                  .where(F.col("_rn") <= F.col("take_n"))
                  .drop("_stratum", "_rn", "count", "take_n"))
else:
    # Fallback: simple uniform sample if no strata columns exist
    frac = min(1.0, SAMPLE_TARGET / df.count()) if df.count() else 1.0
    sampled = df.sample(withReplacement=False, fraction=frac, seed=SEED)

# Save sample (Delta) so you can reuse consistently
sampled.write.mode("overwrite").format("delta").saveAsTable(OUT)

print(f"Stratified sample saved to {OUT}. Rows: {spark.table(OUT).count()}  |  Strata used: {present_strata}")
#  we need to use pandas/scikit-learn for Kmeans, the Pyspark is not working

In [0]:
# --- This is just to check the sample ---
from pyspark.sql import functions as F

SRC = "sc_gold.Features_Table"
OUT = "sc_gold.Features_Table_sampled"

df  = spark.table(SRC)
smp = spark.table(OUT)

print("FULL rows:", df.count(), "| SAMPLE rows:", smp.count())
display(smp.limit(20))  # quick preview


In [0]:
# === K-Means on stratified sample (pandas + scikit) ===
import numpy as np, pandas as pd, sklearn, time
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD

SEED = 42
SAMPLE_TBL = "sc_gold.Features_Table_sampled"   # <- uses your stratified sample

In [0]:
#KMEANS PART 1

# 1) Pull the sample to pandas
pdf = spark.sql(f"SELECT * FROM {SAMPLE_TBL}").toPandas()
print("Sample shape:", pdf.shape)

# 2) Column typing
cat_cols = pdf.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = pdf.select_dtypes(include=["number"]).columns.tolist()

# 3) Preprocess: impute+scale numerics, impute+OHE categoricals (sparse)
numeric_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

categorical_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(**ohe_kwargs))
])

preprocess = ColumnTransformer(
    [("num", numeric_tf, num_cols),
     ("cat", categorical_tf, cat_cols)],
    remainder="drop",
    sparse_threshold=1.0  # keep sparse matrix
)


In [0]:
# KMEANS Part 2

# 4) Transform -> sparse, then reduce dimension (fast & cluster-friendly)
Xs = preprocess.fit_transform(pdf)  # sparse
print("Encoded shape:", Xs.shape)

n_comp = min(30, max(2, Xs.shape[1]-1))  # up to 30 comps, but safe if few features
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)  # dense (n_rows x n_comp)

# scale components (helps K-Means)
X = StandardScaler().fit_transform(X)


In [0]:
# KMEANS PART 3

# 5) Search k quickly (MiniBatchKMeans + sampled silhouette)
k_range = range(2, 9)
scores, best = [], None
t0 = time.perf_counter()

for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    scores.append((k, sil))
    print(f"k={k:2d} | silhouette={sil:.4f} | elapsed={time.perf_counter()-t1:.1f}s")
    if not best or sil > best[1]:
        best = (k, sil, km)

print(f"Total search time: {time.perf_counter()-t0:.1f}s")
best_k, best_sil, best_model = best
print("Best k:", best_k, "| silhouette:", round(best_sil, 4))


In [0]:
# KMEANS PART 4
# 6) Final fit on the sample (already fit above; reuse)
pdf["cluster"] = best_model.predict(X)



In [0]:
#  KMEANS PART 5
# 7) Quick results
print("Cluster sizes:\n", pdf["cluster"].value_counts().sort_index())
peek_cols = [c for c in ["lead_source","is_converted","pais","concessao","instalacao"] if c in pdf.columns]
print(pdf[[*peek_cols, "cluster"]].head(10))



In [0]:
# KMEANS PART 6
# 8) Save clusters back to Delta (sampled table results)
spark.createDataFrame(pdf).write.mode("overwrite").format("delta").saveAsTable("sc_gold.Features_Table_sampled_kmeans")
print("Saved table: sc_gold.Features_Table_sampled_kmeans")

#Best k=2, silhouette≈0.094 → clusters are overlapping/weak. After the tweaks above (merge & bucket modelo, pick one activity style, drop near-constants, SVD to 15–20), re-run the k-scan (2–8). If silhouette stays ~0.1, consider a non-spherical method (UMAP→HDBSCAN) or k-prototypes for mixed data.
#You don’t need a bigger sample yet—the weak silhouette is almost always a feature issue, not a sample-size issue. With 30k rows you already have more than enough signal to see structure if it’s there. I’d first tighten the feature set, then (optionally) bump the sample to ~50–60k only if you want a bit more stability.

In [0]:
# === Replace your cols_to_delete with this one ===
cols_to_delete = [
    # ----------------------------
    # DEAL / POST-OUTCOME LEAKAGE  (*_deals)
    # ----------------------------
    "formulario_contactos",
    "is_converted",
    "tasks_involved",
    "events_involved",
    "calls_involded",
    "tasks_involved_contactos",
    "events_involved_contactos",
    "calls_involded_contactos",
    "activities_involved_contactos",
    "email_opt_out_contactos",
    "salutation_contactos",
    "pais_contactos",
    "formulario_contactos",
    "caracterizacao_contactos",
    "tipo_cliente_propostas",
      "isv_propostas",
    "pintura_propostas",
    "base_tributavel_propostas",
    "legalizacao_propostas",
    "iuc_propostas",
    "preco_total_propostas",
    "iva_propostas",
    "sub_total_com_desconto_propostas",
    "taxa_propostas",
    "total_final_propostas",
    "s_g_p_u__propostas",
     "tasks_only_deals",
    "events_only_deals",
    "calls_only_deals",
    "caracterizacao_contactos",
        "caracterizacao_deals",
    "n__matriculas_por_associar_deals",
    "tipo_cliente_negocio_deals",
    "sales_cycle_duration_deals",
    "calls_involved_contactos",
    "concessao",
    "origem_contactos",
    "proposta_realizada_stage_propostas",
    "valor_base_propostas",
    "preco_base_propostas",
    "age_tier_deals",
    "num_de_propostas_deals",
    "qtd_viaturas_propostas",
    "qtd_viaturas_deals",
    "age_in_days_deals"
]
 


src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")
spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")
spark.sql("DROP VIEW _src")


In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table (pandas + scikit) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load full table to pandas
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Exclude obvious non-features (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    # timestamps/dates
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    # IDs / admin
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    # outcome (not a clustering feature)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

feature_df = pdf.drop(columns=[c for c in exclude if c in pdf.columns])
print("Using features:", feature_df.shape[1], "| Excluded:", len(exclude))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Version-safe OneHotEncoder config
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

# 5) ColumnTransformer
transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions — check your table/filters.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 6) Encode -> SVD to 20 comps -> scale
Xs = preprocess.fit_transform(feature_df)              # usually sparse
n_comp = max(2, min(20, Xs.shape[1] - 1))             # keep <=20 components
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)                              # dense (n_rows x n_comp)
X = StandardScaler().fit_transform(X)                 # scale components

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 7) k-scan (fast): MiniBatchKMeans + sampled silhouette
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.1f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.1f}s")

best_k, best_sil = max(metrics, key=lambda r: r[1])[:2]
print("Best k:", best_k, "| silhouette:", round(best_sil, 4))

# 8) Final fit + labels
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
labels = best_model.predict(X)
pdf["cluster"] = labels
print("Cluster sizes:\n", pdf["cluster"].value_counts().sort_index())

In [0]:
# We really needed to change the features , we tried several times to run the model with no interesting results 
# lets try to use now KMEANS


In [0]:
# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table (pandas + scikit) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load full table to pandas
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Exclude obvious non-features (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    # timestamps/dates
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    # IDs / admin
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    # outcome (not a clustering feature)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

feature_df = pdf.drop(columns=[c for c in exclude if c in pdf.columns])
print("Using features:", feature_df.shape[1], "| Excluded:", len(exclude))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Version-safe OneHotEncoder config
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

# 5) ColumnTransformer
transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions — check your table/filters.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 6) Encode -> SVD to 20 comps -> scale
Xs = preprocess.fit_transform(feature_df)              # usually sparse
n_comp = max(2, min(20, Xs.shape[1] - 1))             # keep <=20 components
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)                              # dense (n_rows x n_comp)
X = StandardScaler().fit_transform(X)                 # scale components

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 7) k-scan (fast): MiniBatchKMeans + sampled silhouette
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.1f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.1f}s")

best_k, best_sil = max(metrics, key=lambda r: r[1])[:2]
print("Best k:", best_k, "| silhouette:", round(best_sil, 4))

# 8) Final fit + labels
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
labels = best_model.predict(X)
pdf["cluster"] = labels
print("Cluster sizes:\n", pdf["cluster"].value_counts().sort_index())

In [0]:
# Is not readable , I will try a different code to understand

In [0]:
# --- Neater K-scan summary ----------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# metrics was built in your loop as: metrics.append((k, sil, inertia, loop_time))
metrics_df = pd.DataFrame(metrics, columns=["k", "silhouette", "inertia", "sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
metrics_df = metrics_df.sort_values(["rank_sil","k"]).reset_index(drop=True)

try:
    display(metrics_df)   # Databricks
except Exception:
    print(metrics_df.to_string(index=False))

In [0]:
# Elbow (inertia) + Silhouette plots
fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)")
ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)")
ax.set_title("Silhouette vs k")
plt.show()

print(f"Best k = {best_k}  |  best silhouette = {best_sil:.4f}")

In [0]:
# --- Final fit & labels (reuse best_k) ---------------------------------------
best_model = MiniBatchKMeans(
    n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED
).fit(X)
labels = best_model.labels_
pdf["cluster"] = labels

# Cluster sizes table (count + %)
sizes = (pdf["cluster"].value_counts()
         .sort_index()
         .rename_axis("cluster")
         .to_frame("count"))
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)

try:
    display(sizes)
except Exception:
    print(sizes)

In [0]:
# --- Per-cluster profile (readable) ------------------------------------------
# Choose which features to summarize (keep it concise).
# If you want "all", leave as None; otherwise list important columns.
focus_numeric = None   # e.g., ["lead_to_deal_days","overall_sales_duration_deals","pvp_propostas"]
focus_categ   = None   # e.g., ["modelo","tipo_cliente_contactos","origem_do_negocio_deals"]

num_cols_all = pdf.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_all = pdf.select_dtypes(include=["object","category","bool"]).columns.tolist()

num_cols = [c for c in (focus_numeric or num_cols_all) if c != "cluster"]
cat_cols = [c for c in (focus_categ or cat_cols_all) if c != "cluster"]

profiles = []

for clu, g in pdf.groupby("cluster", sort=True):
    row = {"cluster": clu, "n": len(g), "pct": round(len(g)*100/len(pdf), 2)}
    # numeric: median (use .median(skipna=True))
    for c in num_cols[:12]:  # cap to ~12 to keep the table readable
        row[f"med_{c}"] = np.nanmedian(g[c].values)
    # categorical: top category + share
    for c in cat_cols[:12]:
        vc = g[c].value_counts(dropna=True, normalize=True)
        if len(vc):
            top_cat = vc.index[0]
            top_pct = round(vc.iloc[0]*100, 1)
            row[f"top_{c}"] = f"{top_cat} ({top_pct}%)"
        else:
            row[f"top_{c}"] = "—"
    profiles.append(row)

profile_df = pd.DataFrame(profiles).sort_values("cluster").reset_index(drop=True)

try:
    display(profile_df)
except Exception:
    # wider print without truncation
    with pd.option_context("display.max_columns", None, "display.width", 200):
        print(profile_df)

In [0]:
# === Step 1: Column selection for clustering ===
# 
# ✅ Drop columns that ended up as "Unknown (~100%)" in Cluster 1:
#   - versao_propostas
#   - combustivel_propostas
#   - origem_do_negocio_deals
#   - motivo_da_perda_deals
#   - campaign_name_campaigns
#
# These don’t help K-Means — they act like noise and just “push” incomplete rows into one group.
# Keep them for other analyses if needed, but not for clustering.
#
# ✅ Keep only business-critical features:
#   - pvp_propostas (price)
#   - modelo
#   - tipo_cliente_contactos
#   - lead_to_deal_days
#   - overall_sales_duration_deals
#   - lead_source
#   - deal_speed_category (optional, since it clearly differentiates Fast vs Slow groups)
#
# This reduced feature set will make clustering more meaningful and interpretable.


In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
# === Replace your cols_to_delete with this one (excludes columns you already removed) ===
cols_to_delete = [
    # ----------------------------
    # LEAKAGE (deal/post-outcome) — info not available at lead time
    # ----------------------------
    "versao_propostas",
    "motivo_da_perda_deals",
    "origem_do_negocio_deals",
    "campaign_name_campaigns",           
   ]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")

In [0]:
# I dont want to drop combustivel but is making a lot of noise , I will replace the unknown by the mode of fuel considering the model of the car but I will also create a new column witht the flag to know is unknown , this is  a good practise for ML to model learns about.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

TBL = "sc_gold.Features_Table"
col_fuel  = "combustivel_propostas"
col_model = "modelo"
flag_col  = f"{col_fuel}_missing"

# Load
df = spark.table(TBL)

# --- BEFORE counts (for sanity) ---
before_unknown = spark.table(TBL)\
    .filter(F.col(col_fuel).isNull() | (F.col(col_fuel) == "Unknown"))\
    .count()
print(f"Unknown/NULL {col_fuel} BEFORE: {before_unknown:,}")

# 1) Flag rows that were missing/Unknown originally
df = df.withColumn(
    flag_col,
    F.when(F.col(col_fuel).isNull() | (F.col(col_fuel) == "Unknown"), 1).otherwise(0).cast(IntegerType())
)

# 2) Per-model MODE (exclude Unknown/NULL)
w = Window.partitionBy(col_model).orderBy(F.desc("cnt"))
mode_per_model = (
    df.filter(F.col(col_fuel).isNotNull() & (F.col(col_fuel) != "Unknown"))
      .groupBy(col_model, col_fuel)
      .count().withColumnRenamed("count", "cnt")
      .withColumn("rn", F.row_number().over(w))
      .filter(F.col("rn") == 1)
      .select(col_model, F.col(col_fuel).alias("mode_fuel"))
)

# 3) Global mode (exclude Unknown/NULL)
glob = (
    df.filter(F.col(col_fuel).isNotNull() & (F.col(col_fuel) != "Unknown"))
      .groupBy(col_fuel).count().orderBy(F.desc("count")).first()
)
global_mode_val = glob[0] if glob else "Unknown"
print(f"Global mode for {col_fuel}: {global_mode_val}")

# 4) Join the per-model mode and impute:
#    - If originally missing/Unknown AND modelo != "Unknown" -> use per-model mode, else global mode
#    - If modelo == "Unknown", keep fuel as "Unknown"
df = df.join(mode_per_model, on=col_model, how="left")

df = df.withColumn(
    col_fuel,
    F.when(
        (F.col(flag_col) == 1) & (F.col(col_model) != "Unknown"),
        F.coalesce(F.col("mode_fuel"), F.lit(global_mode_val))
    ).otherwise(F.col(col_fuel))
).drop("mode_fuel")

# 5) AFTER counts
after_unknown = df.filter(F.col(col_fuel).isNull() | (F.col(col_fuel) == "Unknown")).count()
print(f"Unknown/NULL {col_fuel} AFTER:  {after_unknown:,}")
print(f"Flag column created: {flag_col}")

# 6) Overwrite the table (allow schema change)
df.write.format("delta") \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .option("mergeSchema", "true") \
  .saveAsTable(TBL)

print(f"✅ Overwrote {TBL} with per-model mode imputation (keeping 'Unknown' when modelo='Unknown') and '{flag_col}' flag.")



In [0]:
# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table (pandas + scikit) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load full table to pandas
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Exclude obvious non-features (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    # timestamps/dates
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    # IDs / admin
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    # outcome (not a clustering feature)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

feature_df = pdf.drop(columns=[c for c in exclude if c in pdf.columns])
print("Using features:", feature_df.shape[1], "| Excluded:", len(exclude))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Version-safe OneHotEncoder config
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

# 5) ColumnTransformer
transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions — check your table/filters.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 6) Encode -> SVD to 20 comps -> scale
Xs = preprocess.fit_transform(feature_df)              # usually sparse
n_comp = max(2, min(20, Xs.shape[1] - 1))             # keep <=20 components
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)                              # dense (n_rows x n_comp)
X = StandardScaler().fit_transform(X)                 # scale components

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 7) k-scan (fast): MiniBatchKMeans + sampled silhouette
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.1f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.1f}s")

best_k, best_sil = max(metrics, key=lambda r: r[1])[:2]
print("Best k:", best_k, "| silhouette:", round(best_sil, 4))

# 8) Final fit + labels
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
labels = best_model.predict(X)
pdf["cluster"] = labels
print("Cluster sizes:\n", pdf["cluster"].value_counts().sort_index())

In [0]:
# --- Neater K-scan summary ----------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# metrics was built in your loop as: metrics.append((k, sil, inertia, loop_time))
metrics_df = pd.DataFrame(metrics, columns=["k", "silhouette", "inertia", "sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
metrics_df = metrics_df.sort_values(["rank_sil","k"]).reset_index(drop=True)

try:
    display(metrics_df)   # Databricks
except Exception:
    print(metrics_df.to_string(index=False))

In [0]:
# Elbow (inertia) + Silhouette plots
fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)")
ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)")
ax.set_title("Silhouette vs k")
plt.show()

print(f"Best k = {best_k}  |  best silhouette = {best_sil:.4f}")

In [0]:
# --- Final fit & labels (reuse best_k) ---------------------------------------
best_model = MiniBatchKMeans(
    n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED
).fit(X)
labels = best_model.labels_
pdf["cluster"] = labels

# Cluster sizes table (count + %)
sizes = (pdf["cluster"].value_counts()
         .sort_index()
         .rename_axis("cluster")
         .to_frame("count"))
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)

try:
    display(sizes)
except Exception:
    print(sizes)


In [0]:
# --- Per-cluster profile (readable) ------------------------------------------
# Choose which features to summarize (keep it concise).
# If you want "all", leave as None; otherwise list important columns.
focus_numeric = None   # e.g., ["lead_to_deal_days","overall_sales_duration_deals","pvp_propostas"]
focus_categ   = None   # e.g., ["modelo","tipo_cliente_contactos","origem_do_negocio_deals"]

num_cols_all = pdf.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_all = pdf.select_dtypes(include=["object","category","bool"]).columns.tolist()

num_cols = [c for c in (focus_numeric or num_cols_all) if c != "cluster"]
cat_cols = [c for c in (focus_categ or cat_cols_all) if c != "cluster"]

profiles = []

for clu, g in pdf.groupby("cluster", sort=True):
    row = {"cluster": clu, "n": len(g), "pct": round(len(g)*100/len(pdf), 2)}
    # numeric: median (use .median(skipna=True))
    for c in num_cols[:12]:  # cap to ~12 to keep the table readable
        row[f"med_{c}"] = np.nanmedian(g[c].values)
    # categorical: top category + share
    for c in cat_cols[:12]:
        vc = g[c].value_counts(dropna=True, normalize=True)
        if len(vc):
            top_cat = vc.index[0]
            top_pct = round(vc.iloc[0]*100, 1)
            row[f"top_{c}"] = f"{top_cat} ({top_pct}%)"
        else:
            row[f"top_{c}"] = "—"
    profiles.append(row)

profile_df = pd.DataFrame(profiles).sort_values("cluster").reset_index(drop=True)

try:
    display(profile_df)
except Exception:
    # wider print without truncation
    with pd.option_context("display.max_columns", None, "display.width", 200):
        print(profile_df)

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
FLAG_COL = "deal_before_lead"

df = spark.table(TBL)

# Count how many flagged (==1)
flag_count = df.filter(F.col(FLAG_COL) == 1).count()
total_count = df.count()

print(f"Rows with {FLAG_COL} = 1 : {flag_count:,} out of {total_count:,} ({flag_count*100/total_count:.2f}%)")


In [0]:
# são apenas 7,95% então vamos prosseguir com o uso da moda nestas colunas deal_before_lead .. eu deixei a negativo mas está a fazer noise e os dados começam a não fazer sentido 

from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

TBL       = "sc_gold.Features_Table"
COL_DAYS  = "lead_to_deal_days"
FLAG_COL  = "deal_before_lead"   # <-- if your flag column has a different name, change this

# Load
df = spark.table(TBL)

# Sanity: how many negatives/flagged before
neg_before  = df.filter(F.col(COL_DAYS) < 0).count()
flag_before = df.filter(F.col(FLAG_COL) == 1).count()
print(f"Negatives BEFORE: {neg_before:,}  |  Flagged BEFORE: {flag_before:,}")

# 1) Compute mode from valid (>=0) non-null values
mode_row = (
    df.filter(F.col(COL_DAYS).isNotNull() & (F.col(COL_DAYS) >= 0))
      .groupBy(COL_DAYS).count()
      .orderBy(F.desc("count"))
      .first()
)
mode_val = int(mode_row[0]) if mode_row else 0
print(f"Mode for {COL_DAYS}: {mode_val}")

# 2) Replace where flagged OR negative, else keep original
df = df.withColumn(
    COL_DAYS,
    F.when((F.col(FLAG_COL) == 1) | (F.col(COL_DAYS) < 0), F.lit(mode_val))
     .otherwise(F.col(COL_DAYS))
).withColumn(COL_DAYS, F.col(COL_DAYS).cast(IntegerType()))  # keep dtype consistent

# Sanity: check after
neg_after  = df.filter(F.col(COL_DAYS) < 0).count()
print(f"Negatives AFTER:  {neg_after:,}")

# 3) Overwrite table (no schema change)
df.write.mode("overwrite").saveAsTable(TBL)
print(f"✅ Overwrote {TBL} with mode-imputed {COL_DAYS} for rows flagged by '{FLAG_COL}'.")


In [0]:
# KMEANS AGAIN WITH ALL THOSE CHANGES

# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table (pandas + scikit) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load full table to pandas
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Exclude obvious non-features (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    # timestamps/dates
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    # IDs / admin
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    # outcome (not a clustering feature)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

feature_df = pdf.drop(columns=[c for c in exclude if c in pdf.columns])
print("Using features:", feature_df.shape[1], "| Excluded:", len(exclude))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Version-safe OneHotEncoder config
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

# 5) ColumnTransformer
transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions — check your table/filters.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 6) Encode -> SVD to 20 comps -> scale
Xs = preprocess.fit_transform(feature_df)              # usually sparse
n_comp = max(2, min(20, Xs.shape[1] - 1))             # keep <=20 components
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)                              # dense (n_rows x n_comp)
X = StandardScaler().fit_transform(X)                 # scale components

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 7) k-scan (fast): MiniBatchKMeans + sampled silhouette
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.1f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.1f}s")

best_k, best_sil = max(metrics, key=lambda r: r[1])[:2]
print("Best k:", best_k, "| silhouette:", round(best_sil, 4))

# 8) Final fit + labels
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
labels = best_model.predict(X)
pdf["cluster"] = labels
print("Cluster sizes:\n", pdf["cluster"].value_counts().sort_index())

In [0]:
# --- Neater K-scan summary ----------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# metrics was built in your loop as: metrics.append((k, sil, inertia, loop_time))
metrics_df = pd.DataFrame(metrics, columns=["k", "silhouette", "inertia", "sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
metrics_df = metrics_df.sort_values(["rank_sil","k"]).reset_index(drop=True)

try:
    display(metrics_df)   # Databricks
except Exception:
    print(metrics_df.to_string(index=False))

In [0]:
# Elbow (inertia) + Silhouette plots
fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)")
ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)")
ax.set_title("Silhouette vs k")
plt.show()

print(f"Best k = {best_k}  |  best silhouette = {best_sil:.4f}")

In [0]:
# --- Final fit & labels (reuse best_k) ---------------------------------------
best_model = MiniBatchKMeans(
    n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED
).fit(X)
labels = best_model.labels_
pdf["cluster"] = labels

# Cluster sizes table (count + %)
sizes = (pdf["cluster"].value_counts()
         .sort_index()
         .rename_axis("cluster")
         .to_frame("count"))
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)

try:
    display(sizes)
except Exception:
    print(sizes)

In [0]:
# --- Per-cluster profile (readable) ------------------------------------------
# Choose which features to summarize (keep it concise).
# If you want "all", leave as None; otherwise list important columns.
focus_numeric = None   # e.g., ["lead_to_deal_days","overall_sales_duration_deals","pvp_propostas"]
focus_categ   = None   # e.g., ["modelo","tipo_cliente_contactos","origem_do_negocio_deals"]

num_cols_all = pdf.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_all = pdf.select_dtypes(include=["object","category","bool"]).columns.tolist()

num_cols = [c for c in (focus_numeric or num_cols_all) if c != "cluster"]
cat_cols = [c for c in (focus_categ or cat_cols_all) if c != "cluster"]

profiles = []

for clu, g in pdf.groupby("cluster", sort=True):
    row = {"cluster": clu, "n": len(g), "pct": round(len(g)*100/len(pdf), 2)}
    # numeric: median (use .median(skipna=True))
    for c in num_cols[:12]:  # cap to ~12 to keep the table readable
        row[f"med_{c}"] = np.nanmedian(g[c].values)
    # categorical: top category + share
    for c in cat_cols[:12]:
        vc = g[c].value_counts(dropna=True, normalize=True)
        if len(vc):
            top_cat = vc.index[0]
            top_pct = round(vc.iloc[0]*100, 1)
            row[f"top_{c}"] = f"{top_cat} ({top_pct}%)"
        else:
            row[f"top_{c}"] = "—"
    profiles.append(row)

profile_df = pd.DataFrame(profiles).sort_values("cluster").reset_index(drop=True)

try:
    display(profile_df)
except Exception:
    # wider print without truncation
    with pd.option_context("display.max_columns", None, "display.width", 200):
        print(profile_df)

In [0]:
# === KMEANS AGAIN WITH ALL THOSE CHANGES (exclude combustivel* only for clustering) ===
import time, numpy as np, pandas as pd, sklearn
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load full table to pandas
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Exclude obvious non-features (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    # timestamps/dates
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    # IDs / admin
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    # outcomes
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

# --- EXCLUDE ONLY FOR CLUSTERING (keep in pdf, drop from features) ---
exclude_for_clustering = {"combustivel_propostas", "combustivel_propostas_missing"}

feature_df = pdf.drop(columns=[c for c in (exclude | exclude_for_clustering) if c in pdf.columns])
print("Using features:", feature_df.shape[1], "| Excluded:", len(exclude), "| Dropped-for-clustering:", exclude_for_clustering & set(pdf.columns))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Version-safe OneHotEncoder config
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if version.parse(sklearn.__version__) >= version.parse("1.2") else "sparse"] = True

# 5) ColumnTransformer
transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions — check your table/filters.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 6) Encode -> SVD to 20 comps -> scale
Xs = preprocess.fit_transform(feature_df)              # usually sparse
n_comp = max(2, min(20, Xs.shape[1] - 1))             # keep <=20 components
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)                              # dense (n_rows x n_comp)
X = StandardScaler().fit_transform(X)                 # scale components

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 7) k-scan (fast): MiniBatchKMeans + sampled silhouette
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)


In [0]:
# === Robust k-scan (2..8) + safe display ===
import time, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score

k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()

for k in k_range:
    t1 = time.perf_counter()
    try:
        km = MiniBatchKMeans(
            n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED
        )
        labels = km.fit_predict(X)

        # guard: sometimes k-means collapses to 1 cluster or all labels nan
        uniq = np.unique(labels)
        if uniq.size < 2:
            print(f"⚠️  Skipping k={k}: only {uniq.size} unique label(s).")
            continue

        sil = silhouette_score(
            X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED
        )
        loop_s = time.perf_counter() - t1
        metrics.append((k, float(sil), float(km.inertia_), float(loop_s)))
        print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={loop_s:.2f}s")

    except Exception as e:
        print(f"❌ Failed k={k}: {e}")

print(f"Total k-scan: {time.perf_counter()-t0:.2f}s")

# ---- Build metrics table (if any) ----
metrics_df = pd.DataFrame(metrics, columns=["k","silhouette","inertia","sec_per_fit"])
if metrics_df.empty:
    raise RuntimeError(
        "No valid k results collected. Check earlier messages for 'Failed k=' or 'Skipping k=' "
        "and verify that X has > 1 sample and reasonable variance."
    )

metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
metrics_df = metrics_df.sort_values(["rank_sil","k"]).reset_index(drop=True)
print("\nK-scan metrics:")
print(metrics_df.to_string(index=False))

# ---- Plots ----
fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)"); ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)"); ax.set_title("Silhouette vs k")
plt.show()

# ---- Pick best k safely ----
best_row = metrics_df.loc[metrics_df["silhouette"].idxmax()]
best_k, best_sil = int(best_row["k"]), float(best_row["silhouette"])
print(f"Best k = {best_k}  |  best silhouette = {best_sil:.4f}")

# ---- Final fit + sizes ----
best_model = MiniBatchKMeans(
    n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED
).fit(X)
pdf["cluster"] = best_model.labels_

sizes = (pdf["cluster"].value_counts()
         .sort_index()
         .rename_axis("cluster")
         .to_frame("count"))
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)
print("\nCluster sizes:")
print(sizes.to_string())


In [0]:
# to try to understand the 8% 

# ==== Cluster profiling for current K-Means result (2 clusters) ====
import numpy as np
import pandas as pd

assert "cluster" in pdf.columns, "Run K-Means first to create pdf['cluster']"

# 1) Sizes
sizes = (pdf["cluster"].value_counts()
         .sort_index()
         .rename_axis("cluster")
         .to_frame("count"))
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)

print("\nCluster sizes:")
print(sizes.to_string())
spark.createDataFrame(sizes.reset_index()).show(truncate=False)

# 2) Choose columns to summarize (add/remove as you like)
focus_numeric = [
    "pvp_propostas",
    "overall_sales_duration_deals",
    "lead_to_deal_days",
    "proposals_per_vehicle"
]
focus_categ = [
    "modelo",
    "lead_source",
    "tipo_cliente_contactos",
    "deal_speed_category",
    "Season",
    # we excluded fuel from clustering, but we *do* profile it:
    "combustivel_propostas",
    # optionally:
    "campaign_name_campaigns",
    "origem_do_negocio_deals",
    "motivo_da_perda_deals"
]

num_cols = [c for c in focus_numeric if c in pdf.columns]
cat_cols = [c for c in focus_categ   if c in pdf.columns]

# 3) Per-cluster profile (medians for numeric, top category + % for categorical)
profiles = []
for clu, g in pdf.groupby("cluster", sort=True):
    row = {"cluster": int(clu), "n": len(g), "pct": round(len(g)*100/len(pdf), 2)}
    for c in num_cols:
        row[f"med_{c}"] = float(np.nanmedian(g[c].values))
    for c in cat_cols:
        vc = g[c].value_counts(dropna=True, normalize=True)
        row[f"top_{c}"] = f"{vc.index[0]} ({round(vc.iloc[0]*100,1)}%)" if len(vc) else "—"
    profiles.append(row)

profile_df = pd.DataFrame(profiles).sort_values("cluster").reset_index(drop=True)

print("\nCluster profile (medians + top categories):")
with pd.option_context("display.max_columns", None, "display.width", 220):
    print(profile_df)

spark.createDataFrame(profile_df).show(truncate=False)

# 4) Side-by-side numeric comparison (delta = Cluster1 − Cluster0)
if set([0,1]).issubset(set(pdf["cluster"].unique())) and num_cols:
    piv = (pdf[["cluster"] + num_cols]
           .groupby("cluster").median(numeric_only=True)
           .rename_axis("cluster")
           .sort_index())
    piv_delta = (piv.loc[1] - piv.loc[0]).to_frame("delta_c1_minus_c0").round(2)
    print("\nNumeric medians by cluster:")
    print(piv.round(2).to_string())
    print("\nDelta (Cluster 1 − Cluster 0):")
    print(piv_delta.to_string())
    spark.createDataFrame(piv.reset_index()).show(truncate=False)
    spark.createDataFrame(piv_delta.reset_index().rename(columns={"index":"feature"})).show(truncate=False)

# 5) Categorical lift: which categories are most over-represented in Cluster 1 vs 0
def top_lift(col, topn=8):
    # normalized frequency per cluster
    tab = (pdf.groupby("cluster")[col]
             .value_counts(normalize=True, dropna=True)
             .rename("pct")
             .reset_index())
    c0 = tab[tab["cluster"]==0][["level_1","pct"]].set_index("level_1")
    c1 = tab[tab["cluster"]==1][["level_1","pct"]].set_index("level_1")
    join = c0.join(c1, how="outer", lsuffix="_c0", rsuffix="_c1").fillna(0.0)
    join["lift_c1_vs_c0"] = (join["pct_c1"] + 1e-9) / (join["pct_c0"] + 1e-9)
    return (join.sort_values("lift_c1_vs_c0", ascending=False)
                .head(topn)
                .reset_index()
                .rename(columns={"level_1": col}))

for c in cat_cols:
    try:
        lift = top_lift(c, topn=8)
        print(f"\nTop lift in Cluster 1 vs 0 — {c}:")
        print(lift.to_string(index=False, float_format=lambda x: f"{x:.3f}"))
        spark.createDataFrame(lift).show(truncate=False)
    except Exception as e:
        print(f"(skip lift for {c}: {e})")

# 6) (Optional) persist for BI/SQL
# spark.createDataFrame(profile_df).write.mode("overwrite").saveAsTable("sc_gold.cluster_profile_latest")
# spark.createDataFrame(sizes.reset_index()).write.mode("overwrite").saveAsTable("sc_gold.cluster_sizes_latest")


In [0]:
# === Replace your cols_to_delete with this one ===
#The problem with lead_to_deal_days
#40% nulls → already weak, imputation risks creating artificial patterns.
#Negative values → logically impossible (deal before lead). That indicates data quality issues in the source.
#High correlation with overall_sales_duration_deals → they’re both "time-based" and may overlap in meaning.
#Business interpretation issue → if it doesn’t make sense to the business, clusters built on it won’t be actionable.



cols_to_delete = [
    # --- Columns you no longer want ---
    "combustivel_propostas",
    "lead_to_deal_days",
    "deal_before_lead",
    "lead_to_deal_days_missing",
    "combustivel_propostas_missing",
    "deal_speed_category",
    "modelo",
    

]

src_tbl = "sc_gold.Features_Table"

# 1) Get all columns from current table
all_cols = spark.table(src_tbl).columns

# 2) Keep only those not in cols_to_delete
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting for names

# 3) Snapshot table into a temp view
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

# 4) Recreate the table with only kept columns
spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

# 5) Drop temp view
spark.sql("DROP VIEW _src")

print(f"✅ Table {src_tbl} overwritten with {len(keep_cols)} columns (dropped {len(cols_to_delete)})")


In [0]:
%sql          
SELECT * FROM sc_gold.Features_Table

In [0]:
# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table (pandas + scikit) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load full table to pandas
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Exclude obvious non-features (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    # timestamps/dates
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    # IDs / admin
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    # outcome (not a clustering feature)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

feature_df = pdf.drop(columns=[c for c in exclude if c in pdf.columns])
print("Using features:", feature_df.shape[1], "| Excluded:", len(exclude))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Version-safe OneHotEncoder config
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

# 5) ColumnTransformer
transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions — check your table/filters.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 6) Encode -> SVD to 20 comps -> scale
Xs = preprocess.fit_transform(feature_df)              # usually sparse
n_comp = max(2, min(20, Xs.shape[1] - 1))             # keep <=20 components
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)                              # dense (n_rows x n_comp)
X = StandardScaler().fit_transform(X)                 # scale components

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 7) k-scan (fast): MiniBatchKMeans + sampled silhouette
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.1f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.1f}s")

best_k, best_sil = max(metrics, key=lambda r: r[1])[:2]
print("Best k:", best_k, "| silhouette:", round(best_sil, 4))

# 8) Final fit + labels
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
labels = best_model.predict(X)
pdf["cluster"] = labels
print("Cluster sizes:\n", pdf["cluster"].value_counts().sort_index())

In [0]:
# --- Neater K-scan summary ----------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# metrics was built in your loop as: metrics.append((k, sil, inertia, loop_time))
metrics_df = pd.DataFrame(metrics, columns=["k", "silhouette", "inertia", "sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
metrics_df = metrics_df.sort_values(["rank_sil","k"]).reset_index(drop=True)

try:
    display(metrics_df)   # Databricks
except Exception:
    print(metrics_df.to_string(index=False))


In [0]:

# Elbow (inertia) + Silhouette plots
fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)")
ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)")
ax.set_title("Silhouette vs k")
plt.show()

print(f"Best k = {best_k}  |  best silhouette = {best_sil:.4f}")

In [0]:


# Choose which features to summarize (keep it concise).
# If you want "all", leave as None; otherwise list important columns.
focus_numeric = None   # e.g., ["lead_to_deal_days","overall_sales_duration_deals","pvp_propostas"]
focus_categ   = None   # e.g., ["modelo","tipo_cliente_contactos","origem_do_negocio_deals"]

num_cols_all = pdf.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_all = pdf.select_dtypes(include=["object","category","bool"]).columns.tolist()

num_cols = [c for c in (focus_numeric or num_cols_all) if c != "cluster"]
cat_cols = [c for c in (focus_categ or cat_cols_all) if c != "cluster"]

profiles = []

for clu, g in pdf.groupby("cluster", sort=True):
    row = {"cluster": clu, "n": len(g), "pct": round(len(g)*100/len(pdf), 2)}
    # numeric: median (use .median(skipna=True))
    for c in num_cols[:12]:  # cap to ~12 to keep the table readable
        row[f"med_{c}"] = np.nanmedian(g[c].values)
    # categorical: top category + share
    for c in cat_cols[:12]:
        vc = g[c].value_counts(dropna=True, normalize=True)
        if len(vc):
            top_cat = vc.index[0]
            top_pct = round(vc.iloc[0]*100, 1)
            row[f"top_{c}"] = f"{top_cat} ({top_pct}%)"
        else:
            row[f"top_{c}"] = "—"
    profiles.append(row)

profile_df = pd.DataFrame(profiles).sort_values("cluster").reset_index(drop=True)

try:
    display(profile_df)
except Exception:
    # wider print without truncation
    with pd.option_context("display.max_columns", None, "display.width", 200):
        print(profile_df)

In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

col = "concessao_contactos"

df.select(
    F.count("*").alias("total_rows"),
    F.sum(F.when(F.col(col).isNull(), 1).otherwise(0)).alias("null_count")
).withColumn(
    "null_pct", (F.col("null_count") / F.col("total_rows") * 100).cast("double")
).show(truncate=False)


In [0]:
from pyspark.sql import functions as F

TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

col = "concessao_contactos"

# 1) Find mode (most frequent non-null value)
mode_val = (
    df.filter(F.col(col).isNotNull())
      .groupBy(col).count()
      .orderBy(F.desc("count"))
      .first()
)
mode_str = mode_val[0] if mode_val else None
print(f"Mode for {col}: {mode_str}")

# 2) Replace nulls with mode
if mode_str is not None:
    df = df.withColumn(
        col,
        F.when(F.col(col).isNull(), F.lit(mode_str)).otherwise(F.col(col))
    )

# 3) Overwrite the table
df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(TBL)
print(f"✅ Overwrote {TBL} with nulls in '{col}' replaced by mode = '{mode_str}'")


In [0]:
TBL = "sc_gold.Features_Table"
df = spark.table(TBL)

df.select("modelo").distinct().count()


In [0]:
from pyspark.sql import functions as F

df.groupBy("modelo") \
  .count() \
  .orderBy(F.desc("count")) \
  .show(50, truncate=False)  # change 50 to see more


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

TBL = "sc_gold.Features_Table"
df  = spark.table(TBL)

# --- Normalize once for pattern matching ---
m = F.lower(F.regexp_replace(F.trim(F.col("modelo")), r"\s+", " "))
is_null_or_unknown = m.isNull() | (m == "") | (m == "unknown")

# --- Regex patterns (no \b; use lookarounds to avoid Spark parser issues) ---
# Put bare "ioniq" into EV, but ONLY if not HEV/PHEV (handled via precedence below)
bev_pat     = r"(^ioniq$|nexo|ioniq\s*5(?:\s*n)?|ioniq\s*6|ioniq\s*ev|kona\s*electric|kauai\s*ev|(?<!\w)ev(?!\w)|eléctrico|elétrico|electric|battery)"
phev_pat    = r"(?<!\w)phev(?!\w)|plug[-\s]*in"
hev_pat     = r"(?<!\w)hev(?!\w)|híbrido|hibrido|hybrid"
van_pat     = r"(?:^|[\s-])h1(?:$|\s)|h[-\s]?1\b|h350|\bvan\b|\blugares\b"
suv_pat     = r"(tucson|santa\s*fe|kauai|kona(?!\s*electric)|bayon|ix35|palisade|creta|venue)"
compact_pat = r"(i10|i20|i30|i40|fastback)"

# --- Build the new column with clear precedence ---
# IMPORTANT: PHEV and HEV FIRST, then EV — so "Ioniq HEV" maps to HEV, while bare "Ioniq" maps to EV.
df = df.withColumn(
    "modelo_segment",
    F.when(is_null_or_unknown, F.lit("Unknown"))
     .when(F.regexp_like(m, F.lit(phev_pat)),     F.lit("PHEV"))
     .when(F.regexp_like(m, F.lit(hev_pat)),      F.lit("HEV"))
     .when(F.regexp_like(m, F.lit(bev_pat)),      F.lit("EV"))
     .when(F.regexp_like(m, F.lit(van_pat)),      F.lit("Commercial/Van"))
     .when(F.regexp_like(m, F.lit(suv_pat)),      F.lit("SUV"))
     .when(F.regexp_like(m, F.lit(compact_pat)),  F.lit("Compact/City"))
     .otherwise(F.lit("Other"))
)

# --- Distribution with a proper window (percentages) ---
w_all = Window.orderBy(F.lit(1)).rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

dist = (df.groupBy("modelo_segment")
          .count()
          .withColumn("total", F.sum("count").over(w_all))
          .withColumn("pct", (F.col("count") / F.col("total") * 100))
          .drop("total")
          .orderBy(F.desc("count")))

print("Modelo segment distribution:")
dist.show(truncate=False)

# --- Sanity check: show how top raw model strings mapped ---
print("Top 30 (modelo, modelo_segment) pairs by frequency:")
(df.groupBy("modelo", "modelo_segment")
   .count()
   .orderBy(F.desc("count"))
   .show(30, truncate=False))

# --- Persist (adds the new column to the table) ---
df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(TBL)
print("✅ Added 'modelo_segment' (with plain 'Ioniq' → EV and proper precedence) and overwrote table.")


In [0]:
%sql          
SELECT * FROM sc_gold.Features_Table

In [0]:
# === KMEANS AGAIN (exclude modelo* only for clustering) ===
import time, numpy as np, pandas as pd, sklearn
from packaging import version
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load full table to pandas
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Exclude obvious non-features (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    # timestamps/dates
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    # IDs / admin
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    # outcomes
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

# --- EXCLUDE ONLY FOR CLUSTERING (keep in pdf, drop from features) ---
# Swap from combustivel_* to modelo* as requested
exclude_for_clustering = {"modelo", "modelo_segment"}  # only dropped from features if present

feature_df = pdf.drop(columns=[c for c in (exclude | exclude_for_clustering) if c in pdf.columns])
dropped = (exclude_for_clustering & set(pdf.columns))
print("Using features:", feature_df.shape[1],
      "| Excluded(always):", len(exclude),
      "| Dropped-for-clustering:", dropped if dropped else "{}")

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Version-safe OneHotEncoder config
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if version.parse(sklearn.__version__) >= version.parse("1.2") else "sparse"] = True

# 5) ColumnTransformer
transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions — check your table/filters.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 6) Encode -> SVD to 20 comps -> scale
Xs = preprocess.fit_transform(feature_df)              # usually sparse
n_comp = max(2, min(20, Xs.shape[1] - 1))             # keep <=20 components
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)                              # dense (n_rows x n_comp)
X = StandardScaler().fit_transform(X)                 # scale components

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 7) k-scan (fast): MiniBatchKMeans + sampled silhouette
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.2f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.2f}s")

# 8) Plot + pick best k
metrics_df = pd.DataFrame(metrics, columns=["k","silhouette","inertia","sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
print("\nK-scan metrics:")
print(metrics_df.sort_values(["rank_sil","k"]).to_string(index=False))

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)")
ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)")
ax.set_title("Silhouette vs k")
plt.show()

best_k = int(metrics_df.sort_values(["rank_sil","k"]).iloc[0]["k"])
best_sil = float(metrics_df.sort_values(["rank_sil","k"]).iloc[0]["silhouette"])
print(f"\nBest k = {best_k}  |  best silhouette = {best_sil:.4f}")

# 9) Final fit + labels and basic cluster sizes
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
labels = best_model.labels_
pdf["cluster"] = labels
sizes = pdf["cluster"].value_counts().sort_index().to_frame("count")
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)
print("\nCluster sizes:\n", sizes)


In [0]:
# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table (pandas + scikit) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load full table to pandas
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Exclude obvious non-features (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    # timestamps/dates
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    # IDs / admin
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    # outcome (not a clustering feature)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

feature_df = pdf.drop(columns=[c for c in exclude if c in pdf.columns])
print("Using features:", feature_df.shape[1], "| Excluded:", len(exclude))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Version-safe OneHotEncoder config
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

# 5) ColumnTransformer
transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions — check your table/filters.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 6) Encode -> SVD to 20 comps -> scale
Xs = preprocess.fit_transform(feature_df)              # usually sparse
n_comp = max(2, min(20, Xs.shape[1] - 1))             # keep <=20 components
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)                              # dense (n_rows x n_comp)
X = StandardScaler().fit_transform(X)                 # scale components

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 7) k-scan (fast): MiniBatchKMeans + sampled silhouette
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.1f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.1f}s")

best_k, best_sil = max(metrics, key=lambda r: r[1])[:2]
print("Best k:", best_k, "| silhouette:", round(best_sil, 4))

# 8) Final fit + labels
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
labels = best_model.predict(X)
pdf["cluster"] = labels
print("Cluster sizes:\n", pdf["cluster"].value_counts().sort_index())

In [0]:
# --- Neater K-scan summary ----------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# metrics was built in your loop as: metrics.append((k, sil, inertia, loop_time))
metrics_df = pd.DataFrame(metrics, columns=["k", "silhouette", "inertia", "sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
metrics_df = metrics_df.sort_values(["rank_sil","k"]).reset_index(drop=True)

try:
    display(metrics_df)   # Databricks
except Exception:
    print(metrics_df.to_string(index=False))


In [0]:
# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table (modelo* excluded for clustering) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load table
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Generic exclusions (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

# --- IMPORTANT: drop only for clustering (keep for profiling) ---
exclude_for_clustering = {"modelo", "modelo_segment"}
drop_cols = [c for c in (exclude | exclude_for_clustering) if c in pdf.columns]
feature_df = pdf.drop(columns=drop_cols)

print("Using features:", feature_df.shape[1])
print("Excluded(always):", len(exclude))
print("Dropped-for-clustering:", sorted(exclude_for_clustering & set(pdf.columns)))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Encoders
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 5) Encode -> SVD(<=20) -> scale
Xs = preprocess.fit_transform(feature_df)
n_comp = max(2, min(20, Xs.shape[1] - 1))
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
X = StandardScaler().fit_transform(X)

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 6) k-scan
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.2f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.2f}s")

# 7) Summarize + plots
metrics_df = pd.DataFrame(metrics, columns=["k","silhouette","inertia","sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
print("\nK-scan metrics:")
print(metrics_df.sort_values(["rank_sil","k"]).to_string(index=False))

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)"); ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)"); ax.set_title("Silhouette vs k")
plt.show()

best_row = metrics_df.sort_values(["rank_sil","k"]).iloc[0]
best_k = int(best_row["k"])
best_sil = float(best_row["silhouette"])
print(f"\nBest k = {best_k}  |  best silhouette = {best_sil:.4f}")

# 8) Final fit + cluster sizes
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
pdf["cluster"] = best_model.labels_
sizes = pdf["cluster"].value_counts().sort_index().to_frame("count")
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)
print("\nCluster sizes:\n", sizes)


In [0]:
# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table (concessao_contactos excluded for clustering) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load table
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Generic exclusions (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

# --- IMPORTANT: drop only for clustering (keep for profiling) ---
exclude_for_clustering = {"concessao_contactos"}   # <— your change
drop_cols = [c for c in (exclude | exclude_for_clustering) if c in pdf.columns]
feature_df = pdf.drop(columns=drop_cols)

print("Using features:", feature_df.shape[1])
print("Excluded(always):", len(exclude))
print("Dropped-for-clustering:", sorted(exclude_for_clustering & set(pdf.columns)))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Encoders
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 5) Encode -> SVD(<=20) -> scale
Xs = preprocess.fit_transform(feature_df)
n_comp = max(2, min(20, Xs.shape[1] - 1))
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
X = StandardScaler().fit_transform(X)

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 6) k-scan
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.2f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.2f}s")

# 7) Summarize + plots
metrics_df = pd.DataFrame(metrics, columns=["k","silhouette","inertia","sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
print("\nK-scan metrics:")
print(metrics_df.sort_values(["rank_sil","k"]).to_string(index=False))

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)"); ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)"); ax.set_title("Silhouette vs k")
plt.show()

best_row = metrics_df.sort_values(["rank_sil","k"]).iloc[0]
best_k = int(best_row["k"]); best_sil = float(best_row["silhouette"])
print(f"\nBest k = {best_k}  |  best silhouette = {best_sil:.4f}")

# 8) Final fit + cluster sizes
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
pdf["cluster"] = best_model.labels_
sizes = pdf["cluster"].value_counts().sort_index().to_frame("count")
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)
print("\nCluster sizes:\n", sizes)


In [0]:
# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table 
# (exclude concessao_contactos + modelo_segment only for clustering) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load table
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Generic exclusions (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

# --- IMPORTANT: drop only for clustering (keep for profiling) ---
exclude_for_clustering = {"concessao_contactos", "modelo_segment"}
drop_cols = [c for c in (exclude | exclude_for_clustering) if c in pdf.columns]
feature_df = pdf.drop(columns=drop_cols)

print("Using features:", feature_df.shape[1])
print("Excluded(always):", len(exclude))
print("Dropped-for-clustering:", sorted(exclude_for_clustering & set(pdf.columns)))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Encoders
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 5) Encode -> SVD(<=20) -> scale
Xs = preprocess.fit_transform(feature_df)
n_comp = max(2, min(20, Xs.shape[1] - 1))
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
X = StandardScaler().fit_transform(X)

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 6) k-scan
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.2f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.2f}s")

# 7) Summarize + plots
metrics_df = pd.DataFrame(metrics, columns=["k","silhouette","inertia","sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
print("\nK-scan metrics:")
print(metrics_df.sort_values(["rank_sil","k"]).to_string(index=False))

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)"); ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)"); ax.set_title("Silhouette vs k")
plt.show()

best_row = metrics_df.sort_values(["rank_sil","k"]).iloc[0]
best_k = int(best_row["k"]); best_sil = float(best_row["silhouette"])
print(f"\nBest k = {best_k}  |  best silhouette = {best_sil:.4f}")

# 8) Final fit + cluster sizes
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
pdf["cluster"] = best_model.labels_
sizes = pdf["cluster"].value_counts().sort_index().to_frame("count")
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)
print("\nCluster sizes:\n", sizes)


In [0]:
# === KMEANS (exclude specific categoricals only for clustering) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load table
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Generic exclusions (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

# --- EXCLUDE ONLY FOR CLUSTERING (keep in pdf, drop from features) ---
exclude_for_clustering = {
    "lead_source",
    "tipo_cliente_contactos",
    "concessao_contactos",
    "modelo_segment",
    "Season"
}

drop_cols = [c for c in (exclude | exclude_for_clustering) if c in pdf.columns]
feature_df = pdf.drop(columns=drop_cols)

print("Using features:", feature_df.shape[1])
print("Excluded(always):", len(exclude))
print("Dropped-for-clustering:", sorted(exclude_for_clustering & set(pdf.columns)))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Encoders
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 5) Encode -> SVD(<=20) -> scale
Xs = preprocess.fit_transform(feature_df)
n_comp = max(2, min(20, Xs.shape[1] - 1))
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
X = StandardScaler().fit_transform(X)

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 6) k-scan
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.2f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.2f}s")

# 7) Summarize + plots
metrics_df = pd.DataFrame(metrics, columns=["k","silhouette","inertia","sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
print("\nK-scan metrics:")
print(metrics_df.sort_values(["rank_sil","k"]).to_string(index=False))

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)"); ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)"); ax.set_title("Silhouette vs k")
plt.show()

best_row = metrics_df.sort_values(["rank_sil","k"]).iloc[0]
best_k = int(best_row["k"])
best_sil = float(best_row["silhouette"])
print(f"\nBest k = {best_k}  |  best silhouette = {best_sil:.4f}")

# 8) Final fit + cluster sizes
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
pdf["cluster"] = best_model.labels_
sizes = pdf["cluster"].value_counts().sort_index().to_frame("count")
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)
print("\nCluster sizes:\n", sizes)


In [0]:
# === FULL, SELF-CONTAINED K-Means on sc_gold.Features_Table (modelo* excluded for clustering) ===
import time, numpy as np, pandas as pd, sklearn
from packaging.version import parse as V
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

SEED = 42
SRC_TBL = "sc_gold.Features_Table"

# 1) Load table
pdf = spark.sql(f"SELECT * FROM {SRC_TBL}").toPandas()
print("Full shape:", pdf.shape)

# 2) Generic exclusions (timestamps, IDs/admin, outcomes, near-constants)
name_l = {c: c.lower() for c in pdf.columns}
def has_any(s, toks): return any(t in s for t in toks)

exclude = set()
for c, lc in name_l.items():
    if has_any(lc, ["_time","timestamp"]) or lc.endswith("_at"):
        exclude.add(c)
    if has_any(lc, ["date","data_"]) and ("year" not in lc and "month" not in lc):
        exclude.add(c)
    if has_any(lc, ["_id","id_","uuid","guid","hash","chave","contrato","codigo","code","tylacode","key",
                    "created_by","modified_by","updated_by","owner","gestor","assigned","pipeline","stage_id"]):
        exclude.add(c)
    if lc in ("is_converted","converted","target"):
        exclude.add(c)

# near-constant columns (>=99.5% same value)
nrows = len(pdf)
for c in pdf.columns:
    vc = pdf[c].value_counts(dropna=True)
    if len(vc) and (vc.iloc[0] / max(1, nrows)) >= 0.995:
        exclude.add(c)

# --- IMPORTANT: drop only for clustering (keep for profiling) ---
exclude_for_clustering = {"modelo", "modelo_segment"}
drop_cols = [c for c in (exclude | exclude_for_clustering) if c in pdf.columns]
feature_df = pdf.drop(columns=drop_cols)

print("Using features:", feature_df.shape[1])
print("Excluded(always):", len(exclude))
print("Dropped-for-clustering:", sorted(exclude_for_clustering & set(pdf.columns)))

# 3) Column groups
cat_cols = feature_df.select_dtypes(include=["object","category","bool"]).columns.tolist()
num_cols = feature_df.select_dtypes(include=["number"]).columns.tolist()
print(f"Found {len(num_cols)} numeric, {len(cat_cols)} categorical")

# 4) Encoders
ohe_kwargs = {"handle_unknown": "ignore"}
ohe_kwargs["sparse_output" if V(sklearn.__version__) >= V("1.2") else "sparse"] = True

transformers = []
if num_cols:
    transformers.append(("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols))
if cat_cols:
    transformers.append(("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(**ohe_kwargs))
    ]), cat_cols))
if not transformers:
    raise ValueError("No usable feature columns after exclusions.")

preprocess = ColumnTransformer(transformers, remainder="drop", sparse_threshold=1.0)

# 5) Encode -> SVD(<=20) -> scale
Xs = preprocess.fit_transform(feature_df)
n_comp = max(2, min(20, Xs.shape[1] - 1))
svd = TruncatedSVD(n_components=n_comp, random_state=SEED)
X = svd.fit_transform(Xs)
X = StandardScaler().fit_transform(X)

print(f"Encoded: {Xs.shape}  |  Reduced (SVD): {X.shape}")

# 6) k-scan
k_range = range(2, 9)
metrics = []
t0 = time.perf_counter()
for k in k_range:
    t1 = time.perf_counter()
    km = MiniBatchKMeans(n_clusters=k, batch_size=4096, n_init=20, max_iter=200, random_state=SEED)
    labels = km.fit_predict(X)
    sil = silhouette_score(X, labels, sample_size=min(10000, X.shape[0]), random_state=SEED)
    metrics.append((k, sil, float(km.inertia_), time.perf_counter()-t1))
    print(f"k={k:2d}  sil={sil:.4f}  inertia={km.inertia_:.0f}  loop={metrics[-1][3]:.2f}s")
print(f"Total k-scan: {time.perf_counter()-t0:.2f}s")

# 7) Summarize + plots
metrics_df = pd.DataFrame(metrics, columns=["k","silhouette","inertia","sec_per_fit"])
metrics_df["rank_sil"] = metrics_df["silhouette"].rank(ascending=False).astype(int)
print("\nK-scan metrics:")
print(metrics_df.sort_values(["rank_sil","k"]).to_string(index=False))

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["inertia"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Inertia (lower is better)"); ax.set_title("Elbow curve")
plt.show()

fig, ax = plt.subplots()
ax.plot(metrics_df["k"], metrics_df["silhouette"], marker="o")
ax.set_xlabel("k"); ax.set_ylabel("Silhouette (higher is better)"); ax.set_title("Silhouette vs k")
plt.show()

best_row = metrics_df.sort_values(["rank_sil","k"]).iloc[0]
best_k = int(best_row["k"])
best_sil = float(best_row["silhouette"])
print(f"\nBest k = {best_k}  |  best silhouette = {best_sil:.4f}")

# 8) Final fit + cluster sizes
best_model = MiniBatchKMeans(n_clusters=best_k, batch_size=4096, n_init=50, max_iter=300, random_state=SEED).fit(X)
pdf["cluster"] = best_model.labels_
sizes = pdf["cluster"].value_counts().sort_index().to_frame("count")
sizes["pct"] = (sizes["count"] / len(pdf) * 100).round(2)
print("\nCluster sizes:\n", sizes)


In [0]:
# === Cluster Profiling ===
import numpy as np

# 1) Cluster sizes (already computed, just show again cleanly)
print("Cluster sizes:")
print(sizes)

# 2) Median of numeric variables by cluster
num_summary = (
    pdf.groupby("cluster")[num_cols]
       .median()
       .round(2)
)
print("\nNumeric medians by cluster:")
print(num_summary)

# Also: deltas vs cluster 0 for interpretation
deltas = num_summary.subtract(num_summary.iloc[0], axis=1)
print("\nDeltas vs cluster 0:")
print(deltas)

# 3) Top categories for each categorical variable
def top_categories(df, col, n=3):
    return (
        df.groupby("cluster")[col]
          .apply(lambda x: x.value_counts(normalize=True)
                            .head(n)
                            .round(3)
                            .to_dict())
    )

cat_summary = {}
for c in cat_cols + ["modelo","modelo_segment","lead_source","tipo_cliente_contactos","concessao_contactos"]:
    if c in pdf.columns:  # only those present
        cat_summary[c] = top_categories(pdf, c, n=3)

print("\nTop categories per cluster (showing top 3 per cluster):")
for col, series in cat_summary.items():
    print(f"\n▶ {col}")
    print(series)

# 4) Attach profiling back to cluster sizes
cluster_profile = sizes.join(num_summary, how="left")
print("\nCluster profile:")
print(cluster_profile)

# 5) Optional: visualize numeric medians
import matplotlib.pyplot as plt

for col in num_cols[:10]:  # first 10 numerics
    plt.figure(figsize=(6,3))
    pdf.groupby("cluster")[col].median().plot(kind="bar", title=f"Median {col} by cluster")
    plt.ylabel(col)
    plt.show()


In [0]:
# =========================
# 📊 CLUSTER INTERPRETATION
# =========================
#
# Cluster Sizes:
# - Cluster 0 → 3.35% (~6,100 rows), small minority / outlier.
# - Cluster 1 → 96.65% (~176,400 rows), large majority.
#
# Numeric Medians:
# - Overall Sales Duration (deals):
#     • Cluster 0 = ~530 days (!!)
#     • Cluster 1 = ~35 days
#     → Gap of ~495 days → Cluster 0 = extremely slow-moving deals.
# - Other numeric fields did not show strong separation.
#
# Categorical Profiles:
# - Lead Source:
#     • Both dominated by Facebook & Site Hyundai Portugal.
#     • Cluster 0 has slightly more Site Hyundai (38.6% vs 33.3%).
# - Tipo Cliente (customer type):
#     • Both are mostly Particular (individuals).
#     • Cluster 0 has more ENI customers (13% vs 7.9%).
#     → ENI clients = longer sales cycles.
# - Concessão (dealerships):
#     • Cluster 0 more concentrated in specific dealers 
#       (Entreposto Lisboa, Madeira Motores, etc.).
#     • Suggests dealership process can influence deal duration.
# - PVP Propostas (price bands):
#     • Cluster 0 has higher share of expensive cars (48k+, 40k+).
#     • Cluster 1 covers cheaper ranges (26k, 21k, 38k).
#     → Expensive vehicles take longer to close.
# - Seasonality:
#     • Cluster 0 = more Winter & Spring.
#     • Cluster 1 = more Summer & Autumn.
#     • Secondary effect, but may point to timing differences.
# - Modelo Segment:
#     • Cluster 0 = more SUVs (46%) and EVs (22%).
#     • Cluster 1 = fewer SUVs/EVs, more Compact/City (27%).
#     → Bigger cars = longer decision cycles.
#
# =========================
# 📝 SUMMARY
# =========================
# - Cluster 0 (~3%):
#     • Slow, complex, high-value deals.
#     • SUVs/EVs, higher price points, ENI customers, web leads,
#       and specific dealerships.
#     • Sales cycle ~530 days.
#
# - Cluster 1 (~97%):
#     • Mainstream, faster-moving deals.
#     • Smaller/cheaper cars, mostly individuals.
#     • Sales cycle ~35 days.
#
# Business Implications:
# - Cluster 0 should NOT be discarded as noise → it is a "premium / 
#   complex sales segment". These deals are fewer but higher-value.
# - They require more tailored nurturing and CRM strategies.
# - Cluster 1 is the backbone of the pipeline (fast conversions).


In [0]:
# === Cluster Comparison Report ===
import pandas as pd

# --- 1) Numeric summary ---
num_summary = (
    pdf.groupby("cluster")[num_cols]
       .median()
       .round(2)
       .T  # transpose → vars as rows, clusters as columns
)
num_summary.columns = [f"Cluster {c}" for c in num_summary.columns]

# --- 2) Top categories per categorical column ---
def top_cat_summary(df, col, n=2):
    """Return top n categories with percentages per cluster."""
    out = {}
    for cl, sub in df.groupby("cluster"):
        vc = sub[col].value_counts(normalize=True).head(n).round(3) * 100
        out[f"Cluster {cl}"] = [f"{idx} ({val:.1f}%)" for idx, val in vc.items()]
    return pd.DataFrame.from_dict(out, orient="columns")

cat_tables = {}
for c in ["lead_source", "tipo_cliente_contactos", 
          "concessao_contactos", "pvp_propostas", 
          "Season", "modelo_segment"]:
    if c in pdf.columns:
        cat_tables[c] = top_cat_summary(pdf, c, n=2)

# --- 3) Build report dictionary ---
report = {"Numeric Medians": num_summary}
report.update(cat_tables)

# --- 4) Display nicely ---
for section, table in report.items():
    print(f"\n==== {section} ====")
    display(table)


In [0]:
# === Sales cycle distribution: overall_sales_duration_deals ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

col = "overall_sales_duration_deals"

# 0) Guard
assert col in pdf.columns, f"{col} not found in pdf.columns"
assert "cluster" in pdf.columns, "cluster column not found"

# Keep only valid rows
df_sc = pdf[[col, "cluster"]].dropna().copy()
df_sc[col] = pd.to_numeric(df_sc[col], errors="coerce")
df_sc = df_sc.dropna()

# 1) Summary stats by cluster
def q(s, p): 
    return np.nanpercentile(s, p)

summary = (
    df_sc.groupby("cluster")[col]
         .agg([
             ("n", "count"),
             ("min", "min"),
             ("median", "median"),
             ("mean", "mean"),
             ("p90", lambda s: q(s, 90)),
             ("p95", lambda s: q(s, 95)),
             ("p99", lambda s: q(s, 99)),
             ("max", "max")
         ])
         .round(2)
)
print("=== Sales cycle summary (days) ===")
display(summary)

# 2) Bucketed distribution table (days)
# tweak the bins if you need finer/coarser buckets
bins = [-np.inf, 7, 14, 21, 30, 45, 60, 90, 120, 180, np.inf]
labels = ["0–7","8–14","15–21","22–30","31–45","46–60","61–90","91–120","121–180",">180"]

df_sc["bucket"] = pd.cut(df_sc[col], bins=bins, labels=labels, right=True)

bucket_tab = (
    df_sc.pivot_table(index="bucket", columns="cluster", values=col, aggfunc="size", fill_value=0)
         .assign(All=lambda t: t.sum(axis=1))
)
bucket_tab_pct = (bucket_tab.div(bucket_tab.sum(axis=0), axis=1) * 100).round(2)

print("\n=== Bucket counts (rows = days bucket; cols = cluster) ===")
display(bucket_tab)
print("\n=== Bucket % (rows = days bucket; cols = cluster) ===")
display(bucket_tab_pct)

# Also show compact KPI buckets that business likes
kpi_bins = [0, 30, 60, 90, np.inf]
kpi_labels = ["<=30","31–60","61–90",">90"]
df_sc["kpi_bucket"] = pd.cut(df_sc[col], bins=kpi_bins, labels=kpi_labels, right=True, include_lowest=True)

kpi_tab = (
    df_sc.pivot_table(index="kpi_bucket", columns="cluster", values=col, aggfunc="size", fill_value=0)
         .assign(All=lambda t: t.sum(axis=1))
)
kpi_tab_pct = (kpi_tab.div(kpi_tab.sum(axis=0), axis=1) * 100).round(2)

print("\n=== KPI buckets (counts) ===")
display(kpi_tab)
print("\n=== KPI buckets (%) ===")
display(kpi_tab_pct)

# 3) ECDF (cumulative) plot per cluster
plt.figure(figsize=(7,5))
for cl, sub in df_sc.groupby("cluster"):
    x = np.sort(sub[col].values)
    y = np.arange(1, len(x)+1) / len(x)
    plt.plot(x, y, label=f"Cluster {cl}")
plt.xlabel("Sales cycle (days)")
plt.ylabel("Cumulative share")
plt.title("ECDF of sales cycle by cluster")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# 4) Histogram per cluster, same bins
plt.figure(figsize=(7,5))
for cl, sub in df_sc.groupby("cluster"):
    plt.hist(sub[col].values, bins=bins, alpha=0.5, label=f"Cluster {cl}")
plt.xlabel("Sales cycle (days)")
plt.ylabel("Count")
plt.title("Sales cycle histogram by cluster")
plt.legend()
plt.show()

# 5) Quick text interpretation helpers (optional)
def quick_readout(s):
    return {
        "median": np.median(s),
        "% <=30": (s <= 30).mean()*100,
        "% <=60": (s <= 60).mean()*100,
        "% >90":  (s > 90).mean()*100
    }

print("\n=== Quick readout per cluster ===")
for cl, sub in df_sc.groupby("cluster"):
    stats = quick_readout(sub[col].values)
    print(f"Cluster {cl}: median={stats['median']:.1f}d | <=30d={stats['% <=30']:.1f}% | <=60d={stats['% <=60']:.1f}% | >90d={stats['% >90']:.1f}%")
